In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

from scipy.linalg import norm
import numpy
from scipy.spatial.distance import pdist,cosine,correlation
from scipy.stats import pearsonr

In [3]:
ratings_filtered = pd.read_csv('ratings_data.csv')

# Data Preprocessing

In [4]:
ratings_filtered.groupby('user_id')[['book_id']].count().min()
ratings_filtered.groupby('user_id')[['book_id']].count().max()
ratings_filtered.groupby('user_id')[['book_id']].count().mean()

ratings_filtered.groupby('book_id')[['user_id']].count().min()
ratings_filtered.groupby('book_id')[['user_id']].count().max()
ratings_filtered.groupby('book_id')[['user_id']].count().mean()


ratings_filtered.shape
ratings_filtered.user_id.nunique()
ratings_filtered.book_id.nunique()

book_id    43
dtype: int64

book_id    110
dtype: int64

book_id    82.077457
dtype: float64

user_id    8
dtype: int64

user_id    219
dtype: int64

user_id    26.596676
dtype: float64

(153649, 4)

1872

5777

In [5]:
r_temp = ratings_filtered.copy(deep=True)

In [6]:
ratings_filtered.head()
ratings_filtered.shape

,Unnamed: 0,user_id,book_id,rating
0,1510,103,867,5
1,1512,103,287,5
2,1513,103,2860,5
3,1514,103,397,4
4,1605,103,2027,5


(153649, 4)

In [7]:
ratings_filtered['test'] = 0
ratings_filtered.sort_values(by='user_id', inplace=True)

# user similarity
import time

start = time.time()


c = 0
u = 0
for index, row in ratings_filtered.iterrows():
    if(c==0):
        row['test'] = 1
        c += 1
        u = row['user_id']
    else:
        if(row['user_id']!=u):
            c += 1
            u = row['user_id']
            row['test'] = 1
    if(c==500):
        break
    

end = time.time()
print(end - start)

train = ratings_filtered[ratings_filtered['test']==0]
test = ratings_filtered[ratings_filtered['test']==1]

train.shape
test.shape

5.9505345821380615


(153149, 5)

(500, 5)

In [8]:
train.groupby('user_id')[['book_id']].count().min()
train.groupby('user_id')[['book_id']].count().max()
train.groupby('user_id')[['book_id']].count().mean()

train.groupby('book_id')[['user_id']].count().min()
train.groupby('book_id')[['user_id']].count().max()
train.groupby('book_id')[['user_id']].count().mean()

book_id    42
dtype: int64

book_id    109
dtype: int64

book_id    81.810363
dtype: float64

user_id    7
dtype: int64

user_id    219
dtype: int64

user_id    26.510126
dtype: float64

# Collaborative Filtering

In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

from scipy.linalg import norm
import numpy
from scipy.spatial.distance import pdist,cosine,correlation
from scipy.stats import pearsonr

In [12]:
user_item = pd.pivot_table(train, index='user_id', columns='book_id', values='rating')

In [15]:
user_item.shape
train.shape
test.shape

(1872, 5777)

(153149, 5)

(500, 5)

In [16]:
user_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1872 entries, 67 to 53412
Columns: 5777 entries, 44 to 9995
dtypes: float64(5777)
memory usage: 82.5 MB


In [17]:
user_item = user_item.add_prefix('book_')
user_item = user_item.set_index('user_'+user_item.index.astype(str))
user_item.head()

book_id,book_44,book_68,book_74,book_77,book_78,book_79,book_82,book_83,book_84,book_86,...,book_9907,book_9922,book_9923,book_9927,book_9938,book_9957,book_9961,book_9966,book_9978,book_9995
user_67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_135,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
data = user_item.copy(deep= True)

In [19]:
data.head()

book_id,book_44,book_68,book_74,book_77,book_78,book_79,book_82,book_83,book_84,book_86,...,book_9907,book_9922,book_9923,book_9927,book_9938,book_9957,book_9961,book_9966,book_9978,book_9995
user_67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_135,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cosine Similarity

In [20]:
from scipy.linalg import norm
import numpy
from scipy.spatial.distance import pdist,cosine,correlation

# user similarity
import time

start = time.time()

m = data.shape[0]
a = numpy.zeros(shape=(len(user_item.index),len(user_item.index)))
df = pd.DataFrame(a,columns=user_item.index.values)
df.set_index(user_item.index.values, inplace=True)

mask = numpy.isfinite(data)
# k = 0
for i in range(m-1):
    print('*******************************user number:'+str(i)+'*****************************************')
    for j in range(i + 1, m):
        curr = numpy.logical_and(mask.iloc[i], mask.iloc[j])
        if(curr.nunique()!=1): 
            u = data.iloc[i][curr]
            v = data.iloc[j][curr]
            df.iloc[i][j] = 1 - cosine(u,v)
            
end = time.time()
print(end - start)

*******************************user number:0*****************************************
*******************************user number:1*****************************************
*******************************user number:2*****************************************
*******************************user number:3*****************************************
*******************************user number:4*****************************************
*******************************user number:5*****************************************
*******************************user number:6*****************************************
*******************************user number:7*****************************************
*******************************user number:8*****************************************
*******************************user number:9*****************************************
*******************************user number:10*****************************************
*******************************user number:11********

*******************************user number:95*****************************************
*******************************user number:96*****************************************
*******************************user number:97*****************************************
*******************************user number:98*****************************************
*******************************user number:99*****************************************
*******************************user number:100*****************************************
*******************************user number:101*****************************************
*******************************user number:102*****************************************
*******************************user number:103*****************************************
*******************************user number:104*****************************************
*******************************user number:105*****************************************
*******************************user n

*******************************user number:189*****************************************
*******************************user number:190*****************************************
*******************************user number:191*****************************************
*******************************user number:192*****************************************
*******************************user number:193*****************************************
*******************************user number:194*****************************************
*******************************user number:195*****************************************
*******************************user number:196*****************************************
*******************************user number:197*****************************************
*******************************user number:198*****************************************
*******************************user number:199*****************************************
*******************************u

*******************************user number:283*****************************************
*******************************user number:284*****************************************
*******************************user number:285*****************************************
*******************************user number:286*****************************************
*******************************user number:287*****************************************
*******************************user number:288*****************************************
*******************************user number:289*****************************************
*******************************user number:290*****************************************
*******************************user number:291*****************************************
*******************************user number:292*****************************************
*******************************user number:293*****************************************
*******************************u

*******************************user number:377*****************************************
*******************************user number:378*****************************************
*******************************user number:379*****************************************
*******************************user number:380*****************************************
*******************************user number:381*****************************************
*******************************user number:382*****************************************
*******************************user number:383*****************************************
*******************************user number:384*****************************************
*******************************user number:385*****************************************
*******************************user number:386*****************************************
*******************************user number:387*****************************************
*******************************u

...

*******************************user number:1499*****************************************
*******************************user number:1500*****************************************
*******************************user number:1501*****************************************
*******************************user number:1502*****************************************
*******************************user number:1503*****************************************
*******************************user number:1504*****************************************
*******************************user number:1505*****************************************
*******************************user number:1506*****************************************
*******************************user number:1507*****************************************
*******************************user number:1508*****************************************
*******************************user number:1509*****************************************
*********************

*******************************user number:1592*****************************************
*******************************user number:1593*****************************************
*******************************user number:1594*****************************************
*******************************user number:1595*****************************************
*******************************user number:1596*****************************************
*******************************user number:1597*****************************************
*******************************user number:1598*****************************************
*******************************user number:1599*****************************************
*******************************user number:1600*****************************************
*******************************user number:1601*****************************************
*******************************user number:1602*****************************************
*********************

*******************************user number:1685*****************************************
*******************************user number:1686*****************************************
*******************************user number:1687*****************************************
*******************************user number:1688*****************************************
*******************************user number:1689*****************************************
*******************************user number:1690*****************************************
*******************************user number:1691*****************************************
*******************************user number:1692*****************************************
*******************************user number:1693*****************************************
*******************************user number:1694*****************************************
*******************************user number:1695*****************************************
*********************

*******************************user number:1778*****************************************
*******************************user number:1779*****************************************
*******************************user number:1780*****************************************
*******************************user number:1781*****************************************
*******************************user number:1782*****************************************
*******************************user number:1783*****************************************
*******************************user number:1784*****************************************
*******************************user number:1785*****************************************
*******************************user number:1786*****************************************
*******************************user number:1787*****************************************
*******************************user number:1788*****************************************
*********************

5796.056277036667


In [22]:
df.head()
df.isnull().values.any()

,user_67,user_103,user_128,user_135,user_137,user_149,user_174,user_245,user_254,user_326,...,user_53280,user_53287,user_53298,user_53306,user_53319,user_53325,user_53341,user_53387,user_53398,user_53412
user_67,0.0,0.947334,0.985941,0.000000,0.0,0.000000,0.948417,0.994937,0.000000,1.000000,...,0.000000,0.962140,1.000000,0.948174,1.000000,1.000000,0.000000,0.974835,1.000000,0.000000
user_103,0.0,0.000000,0.936244,1.000000,0.0,0.957510,0.924945,1.000000,0.929152,0.968730,...,0.993884,0.880705,0.999512,1.000000,1.000000,0.989949,0.000000,1.000000,1.000000,0.899401
user_128,0.0,0.000000,0.000000,0.950542,0.0,0.000000,0.925996,1.000000,1.000000,0.924160,...,0.952579,0.994937,0.980581,0.000000,0.964901,0.993884,0.000000,0.999512,0.000000,0.000000
user_135,0.0,0.000000,0.000000,0.000000,1.0,0.983015,0.000000,0.000000,0.970143,0.870022,...,0.977802,1.000000,0.971444,1.000000,0.832050,1.000000,0.979911,1.000000,0.994692,1.000000
user_137,0.0,0.000000,0.000000,0.000000,0.0,0.940582,1.000000,0.954792,0.989949,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.968277,1.000000,0.932278,1.000000,0.909137,0.965300


False

# Correlation

In [23]:
from scipy.linalg import norm
import numpy
from scipy.spatial.distance import pdist,cosine,correlation

# user similarity
import time

start = time.time()

m = data.shape[0]
a = numpy.zeros(shape=(len(user_item.index),len(user_item.index)))

df_corr = pd.DataFrame(a,columns=user_item.index.values)
df_corr.set_index(user_item.index.values, inplace=True)
mask = numpy.isfinite(data)
for i in range(m-1):
    print('*******************************user number:'+str(i)+'*****************************************')
    for j in range(i + 1, m):
        curr = numpy.logical_and(mask.iloc[i], mask.iloc[j])
        if(curr.nunique()!=1): 
            u = data.iloc[i][curr]
            v = data.iloc[j][curr]
            df_corr.iloc[i][j] = 1-correlation(u, v)

end = time.time()
print(end - start)

*******************************user number:0*****************************************
*******************************user number:1*****************************************
*******************************user number:2*****************************************
*******************************user number:3*****************************************
*******************************user number:4*****************************************
*******************************user number:5*****************************************
*******************************user number:6*****************************************
*******************************user number:7*****************************************
*******************************user number:8*****************************************
*******************************user number:9*****************************************
*******************************user number:10*****************************************
*******************************user number:11********

*******************************user number:95*****************************************
*******************************user number:96*****************************************
*******************************user number:97*****************************************
*******************************user number:98*****************************************
*******************************user number:99*****************************************
*******************************user number:100*****************************************
*******************************user number:101*****************************************
*******************************user number:102*****************************************
*******************************user number:103*****************************************
*******************************user number:104*****************************************
*******************************user number:105*****************************************
*******************************user n

*******************************user number:189*****************************************
*******************************user number:190*****************************************
*******************************user number:191*****************************************
*******************************user number:192*****************************************
*******************************user number:193*****************************************
*******************************user number:194*****************************************
*******************************user number:195*****************************************
*******************************user number:196*****************************************
*******************************user number:197*****************************************
*******************************user number:198*****************************************
*******************************user number:199*****************************************
*******************************u

*******************************user number:283*****************************************
*******************************user number:284*****************************************
*******************************user number:285*****************************************
*******************************user number:286*****************************************
*******************************user number:287*****************************************
*******************************user number:288*****************************************
*******************************user number:289*****************************************
*******************************user number:290*****************************************
*******************************user number:291*****************************************
*******************************user number:292*****************************************
*******************************user number:293*****************************************
*******************************u

*******************************user number:377*****************************************
*******************************user number:378*****************************************
*******************************user number:379*****************************************
*******************************user number:380*****************************************
*******************************user number:381*****************************************
*******************************user number:382*****************************************
*******************************user number:383*****************************************
*******************************user number:384*****************************************
*******************************user number:385*****************************************
*******************************user number:386*****************************************
*******************************user number:387*****************************************
*******************************u

...

*******************************user number:1406*****************************************
*******************************user number:1407*****************************************
*******************************user number:1408*****************************************
*******************************user number:1409*****************************************
*******************************user number:1410*****************************************
*******************************user number:1411*****************************************
*******************************user number:1412*****************************************
*******************************user number:1413*****************************************
*******************************user number:1414*****************************************
*******************************user number:1415*****************************************
*******************************user number:1416*****************************************
*********************

*******************************user number:1499*****************************************
*******************************user number:1500*****************************************
*******************************user number:1501*****************************************
*******************************user number:1502*****************************************
*******************************user number:1503*****************************************
*******************************user number:1504*****************************************
*******************************user number:1505*****************************************
*******************************user number:1506*****************************************
*******************************user number:1507*****************************************
*******************************user number:1508*****************************************
*******************************user number:1509*****************************************
*********************

*******************************user number:1592*****************************************
*******************************user number:1593*****************************************
*******************************user number:1594*****************************************
*******************************user number:1595*****************************************
*******************************user number:1596*****************************************
*******************************user number:1597*****************************************
*******************************user number:1598*****************************************
*******************************user number:1599*****************************************
*******************************user number:1600*****************************************
*******************************user number:1601*****************************************
*******************************user number:1602*****************************************
*********************

*******************************user number:1685*****************************************
*******************************user number:1686*****************************************
*******************************user number:1687*****************************************
*******************************user number:1688*****************************************
*******************************user number:1689*****************************************
*******************************user number:1690*****************************************
*******************************user number:1691*****************************************
*******************************user number:1692*****************************************
*******************************user number:1693*****************************************
*******************************user number:1694*****************************************
*******************************user number:1695*****************************************
*********************

*******************************user number:1778*****************************************
*******************************user number:1779*****************************************
*******************************user number:1780*****************************************
*******************************user number:1781*****************************************
*******************************user number:1782*****************************************
*******************************user number:1783*****************************************
*******************************user number:1784*****************************************
*******************************user number:1785*****************************************
*******************************user number:1786*****************************************
*******************************user number:1787*****************************************
*******************************user number:1788*****************************************
*********************

In [24]:
df_corr.isnull().values.any()
df_corr.fillna(0,inplace=True)
df_corr.head()
df_corr.isnull().values.any()

True

,user_67,user_103,user_128,user_135,user_137,user_149,user_174,user_245,user_254,user_326,...,user_53280,user_53287,user_53298,user_53306,user_53319,user_53325,user_53341,user_53387,user_53398,user_53412
user_67,0.0,-0.229416,0.426401,0.000000,0.0,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.57735,0.000000,0.220863,0.0,0.0,0.000000,0.0,0.00000,0.000000
user_103,0.0,0.000000,0.467707,0.000000,0.0,0.000000,-0.492410,0.000000,-0.298279,0.311376,...,0.000000,0.00000,1.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,-0.500000
user_128,0.0,0.000000,0.000000,0.866025,0.0,0.000000,0.395285,0.000000,1.000000,-0.583333,...,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.00000,0.000000
user_135,0.0,0.000000,0.000000,0.000000,0.0,0.242536,0.000000,0.000000,0.000000,-1.000000,...,0.944911,0.00000,0.670166,0.000000,0.0,0.0,0.745356,0.0,1.00000,0.000000
user_137,0.0,0.000000,0.000000,0.000000,0.0,0.500000,0.000000,-0.174078,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,-1.0,0.0,-0.294174,0.0,-0.19245,0.242536


False

# Testing

In [87]:
train.shape
test.shape
test.head()

(153149, 5)

(65, 3)

,user_id,book_id,rating
id,,,
0,137,2123,2
1,516,416,4
2,524,3309,3
3,681,174,4
4,1009,101,4


In [89]:
# book_ids which are rated by user1, book_ids not rated by user1 but rated by others
test['pred_rating_cos'] = 0.0
test['pred_rating_corr'] = 0.0
test['mean_rating'] = 0.0

for index, row in test.iterrows():
    print('*********************user:'+str(index)+'***************************')
    user = row['user_id']
    user_col = 'user_'+str(int(user))
    
    item = row['book_id']
    
    user_list = train[train['book_id']==item]['user_id'].unique()
    train_temp = train[(train.user_id.map(lambda x: x in user_list))]
    
    df_user = train[train['user_id']==user]
    df_all = train_temp[train_temp['user_id']!=user]
    
    test['mean_rating'][index] = df_user['rating'].mean()
    
    df_all['mean_rating'] = 0.0
    user_means = {}
    
    for u in df_all['user_id'].unique():
        user_means[u] = df_all[df_all['user_id'] == u]['rating'].mean()
    
    df_all['mean_rating'] = df_all.apply(lambda x: user_means[x['user_id']], axis=1)
    df_all['adjusted_rating'] = df_all['rating'] - df_all['mean_rating']
    df_all = df_all[df_all['book_id']==item]
    
    df_all['user_id'] = df_all['user_id'].map(lambda x: 'user_'+str(int(x)))
    
    df_all['similarity_cosine'] = df_all.user_id.apply(lambda x: df.loc[user_col,x] if df.loc[user_col,x]!=0 else df.loc[x,user_col])   
    df_all['similarity_correlation'] = df_all.user_id.apply(lambda x: df_corr.loc[user_col,x] if df_corr.loc[user_col,x]!=0 else df_corr.loc[x,user_col])
    
    # weighted similarity
    cos_sum = df_all['similarity_cosine'].abs().sum()
    corr_sum = df_all['similarity_correlation'].abs().sum()
    df_all['similarity_cosine'] = df_all['similarity_cosine'].map(lambda x: float(x)/cos_sum)
    df_all['similarity_correlation'] = df_all['similarity_correlation'].map(lambda x: float(x)/corr_sum)
    
    rating_list_cos = [row['similarity_cosine']*row['adjusted_rating'] for index, row in df_all.iterrows()]
    rating_list_corr = [row['similarity_correlation']*row['adjusted_rating'] for index, row in df_all.iterrows()]

    test['pred_rating_cos'][index] = test['mean_rating'][index]+sum(rating_list_cos)
    test['pred_rating_corr'][index] = test['mean_rating'][index]+sum(rating_list_corr)

test['error_cos'] = abs(test['pred_rating_cos'] - test['rating'])
test['error_corr'] = abs(test['pred_rating_corr'] - test['rating'])

*********************user:0***************************
*********************user:1***************************
*********************user:2***************************
*********************user:3***************************
*********************user:4***************************
*********************user:5***************************
*********************user:6***************************
*********************user:7***************************
*********************user:8***************************
*********************user:9***************************
*********************user:10***************************
*********************user:11***************************
*********************user:12***************************
*********************user:13***************************
*********************user:14***************************
*********************user:15***************************
*********************user:16***************************
*********************user:17***************************
**

In [90]:
test['error_cos_sq'] = test['error_cos']*test['error_cos']
test['error_corr_sq'] = test['error_corr']*test['error_corr']

In [91]:
train.head()

,Unnamed: 0,user_id,book_id,rating,test
id,,,,,
1037684,25233,67,1523,3,0
1024564,24969,67,1481,3,0
1024562,24968,67,564,3,0
1024560,24967,67,435,4,0
968609,23514,67,106,5,0


In [92]:
print('MAE with mean rating is:'+str( float( (abs( test['mean_rating']-test['rating'] )).sum() ) /test.shape[0]))
print('MAE with Cosine Similarity is:'+str(float(test['error_cos'].sum())/test.shape[0]))
print('MAE with Correlation is:'+str(float(test['error_corr'].sum())/test.shape[0]))
print('\n')

print('MSE with mean rating is:'+str( float( (( test['mean_rating']-test['rating'] )*( test['mean_rating']-test['rating'] )).sum() ) /test.shape[0]))
print('MSE with Cosine Similarity is:'+str(float(test['error_cos_sq'].sum())/test.shape[0]))
print('MSE with Correlation is:'+str(float(test['error_corr_sq'].sum())/test.shape[0]))     

MAE with mean rating is:0.6712791682489818
MAE with Cosine Similarity is:0.6518389959581088
MAE with Correlation is:0.6745857155526924


MSE with mean rating is:0.688032817965408
MSE with Cosine Similarity is:0.6850339756426674
MSE with Correlation is:0.6951832882672123


# Recommendations

In [9]:
# take user input
n = 9554
user = 'user_'+str(n)
print(user)

user_9554


In [10]:
train[train['user_id'] == n]['user_id'].unique()

array([9554], dtype=int64)

In [12]:
# book_ids which are rated by user, book_ids not rated by user but rated by others

# mean rating for user1
user_means = {}
user_means[n] = train[train['user_id'] == n]['rating'].mean()

df_temp  = pd.DataFrame()



for i in list(sim_user_n_u.loc[user]): #train[train['user_id'] != n]['user_id'].unique(): #
    u = int(str(i).split('_')[1]) #i
    df1 = train[train['user_id']==u]
    df2 = train[train['user_id']==n]
    df3 = df1[~df1.isin(df2)].dropna(how = 'all')
    df_temp = pd.concat([df_temp, df3])
    user_means[u] = train[train['user_id'] == u]['rating'].mean()
    
df_temp['mean_rating'] = df_temp.apply(lambda row: user_means[row['user_id']], axis=1)

In [13]:
df_temp['adjusted_rating'] = df_temp['rating']-df_temp['mean_rating']
df_temp['similarity'] = df_temp.user_id.apply(lambda x: df.loc[user,'user_'+str(x)] if df.loc[user,'user_'+str(x)]!=0 else df.loc['user_'+str(x),user])
books_rating_diff = {}
for b in df_temp.book_id.unique():
    df_books = df_temp[df_temp['book_id']==b].copy(deep=True)
    if(df_books.shape[0]>1):
        df_books['weight'] = df_books['similarity']/df_books.similarity.abs().sum()
        df_books['rating_weight'] = df_books['adjusted_rating']*df_books['weight']
        books_rating_diff[b] = df_books.rating_weight.sum()
    

In [14]:
recommend_books = pd.DataFrame(columns=['book_id','rating_difference'])
recommend_books['book_id'] = [key for key in books_rating_diff.keys()]
recommend_books['rating_difference'] = recommend_books['book_id'].map(lambda x: books_rating_diff[x])
recommend_books.sort_values(by='rating_difference', ascending=False, inplace=True)

In [15]:
books = pd.read_csv('books.csv')
recommend_df = pd.merge(recommend_books, books, on='book_id', how='inner')[['book_id','rating_difference','title','authors']]

In [19]:
print("Recommend following books:")
recommend_df[:5]

,title_x,authors_x,cluster_num
0,The Complete Calvin and Hobbes,Bill Watterson,7
1,Where the Heart Is,Billie Letts,7
2,A Tree Grows in Brooklyn,Betty Smith,7
3,It's a Magical World: A Calvin and Hobbes Coll...,Bill Watterson,7
4,The Calvin and Hobbes Lazy Sunday Book,Bill Watterson,7
